# Fase 4: Temperature Scaling para Calibración de Probabilidades

**Objetivo**: Calibrar las probabilidades del modelo para que reflejen la confianza real (p=0.8 → 80% accuracy).

**Pipeline**:
1. Extraer predicciones del baseline en val_calib
2. Ajustar temperatura T minimizando NLL
3. Aplicar T en val_eval
4. Evaluar mejora en calibración (ECE, Brier, NLL, Reliability Diagrams)

## 1. Imports y Configuración

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import yaml
from scipy.optimize import minimize
from sklearn.metrics import brier_score_loss
import warnings
warnings.filterwarnings('ignore')

OUTPUT_DIR = Path('./outputs/temperature_scaling')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

CONFIG = {
    'seed': 42,
    'iou_matching': 0.5,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'categories': ['person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'traffic light', 'traffic sign']
}

torch.manual_seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])

print(f"Device: {CONFIG['device']}")
print(f"Output: {OUTPUT_DIR}")

## 2. Cargar Modelo Baseline

In [ ]:
from groundingdino.util.inference import load_model, load_image, predict
from groundingdino.util import box_ops

model_config = '/opt/program/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
model_weights = '/opt/program/GroundingDINO/weights/groundingdino_swint_ogc.pth'

model = load_model(model_config, model_weights)
model.to(CONFIG['device'])
model.eval()

TEXT_PROMPT = '. '.join(CONFIG['categories']) + '.'
print(f"Modelo cargado en {CONFIG['device']}")
print(f"Prompt: {TEXT_PROMPT}")

## 3. Inferencia en val_calib y Matching con GT

In [ ]:
from pycocotools.coco import COCO
import torchvision

def compute_iou(box1, box2):
    """Calcula IoU entre dos boxes [x1, y1, x2, y2]"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0

def normalize_label(label):
    """Normaliza label del modelo a categoría canónica"""
    synonyms = {'bike': 'bicycle', 'motorbike': 'motorcycle', 'pedestrian': 'person', 
                'stop sign': 'traffic sign', 'red light': 'traffic light'}
    label_lower = label.lower().strip()
    if label_lower in synonyms:
        return synonyms[label_lower]
    for cat in CONFIG['categories']:
        if cat in label_lower:
            return cat
    return label_lower

def run_inference_on_image(model, image_path, text_prompt, conf_thresh, device):
    """Ejecuta inferencia en una imagen"""
    image_source, image = load_image(str(image_path))
    boxes, logits, phrases = predict(model, image, text_prompt, conf_thresh, 0.25, device)
    
    if len(boxes) == 0:
        return []
    
    h, w = image_source.shape[:2]
    boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.tensor([w, h, w, h])
    
    detections = []
    for box, logit, phrase in zip(boxes_xyxy.cpu().numpy(), logits.cpu().numpy(), phrases):
        cat = normalize_label(phrase)
        if cat in CONFIG['categories']:
            detections.append({
                'bbox': box.tolist(),
                'score': float(logit),
                'logit': float(logit),
                'category': cat
            })
    
    boxes_t = torch.tensor([d['bbox'] for d in detections])
    scores_t = torch.tensor([d['score'] for d in detections])
    keep = torchvision.ops.nms(boxes_t, scores_t, 0.65)
    
    return [detections[i] for i in keep.numpy()]

def match_predictions_to_gt(predictions, gt_anns, iou_thresh):
    """Asigna TP/FP a predicciones según matching con GT"""
    gt_matched = set()
    results = []
    
    cat_to_id = {cat: i+1 for i, cat in enumerate(CONFIG['categories'])}
    id_to_cat = {v: k for k, v in cat_to_id.items()}
    
    for pred in predictions:
        pred_cat = pred['category']
        pred_box = pred['bbox']
        
        best_iou = 0
        best_gt_idx = -1
        
        for idx, gt in enumerate(gt_anns):
            if idx in gt_matched:
                continue
            
            gt_cat_id = gt['category_id']
            gt_cat = id_to_cat.get(gt_cat_id, '')
            
            if gt_cat != pred_cat:
                continue
            
            gt_box = gt['bbox']
            gt_box_xyxy = [gt_box[0], gt_box[1], gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]]
            iou = compute_iou(pred_box, gt_box_xyxy)
            
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = idx
        
        if best_iou >= iou_thresh:
            gt_matched.add(best_gt_idx)
            is_tp = 1
        else:
            is_tp = 0
        
        results.append({
            'logit': pred['logit'],
            'score': pred['score'],
            'category': pred_cat,
            'is_tp': is_tp,
            'iou': best_iou
        })
    
    return results

print("Funciones de inferencia y matching definidas")

In [ ]:
val_calib_json = Path('../data/bdd100k_coco/val_calib.json')
image_dir = Path('../data/bdd100k/bdd100k/bdd100k/images/100k/val')

coco_calib = COCO(str(val_calib_json))
img_ids = coco_calib.getImgIds()

print(f"Procesando {len(img_ids)} imágenes de val_calib...")

calib_data = []

for img_id in tqdm(img_ids[:500]):  # Procesar primeras 500 para calibración
    img_info = coco_calib.loadImgs(img_id)[0]
    img_path = image_dir / img_info['file_name']
    
    if not img_path.exists():
        continue
    
    predictions = run_inference_on_image(model, img_path, TEXT_PROMPT, 0.25, CONFIG['device'])
    
    gt_anns = coco_calib.loadAnns(coco_calib.getAnnIds(imgIds=img_id))
    
    matched = match_predictions_to_gt(predictions, gt_anns, CONFIG['iou_matching'])
    calib_data.extend(matched)

calib_df = pd.DataFrame(calib_data)
calib_df.to_csv(OUTPUT_DIR / 'calib_detections.csv', index=False)

print(f"\nTotal detecciones: {len(calib_df)}")
print(f"TP: {calib_df['is_tp'].sum()}, FP: {len(calib_df) - calib_df['is_tp'].sum()}")
print(f"Guardado en: {OUTPUT_DIR / 'calib_detections.csv'}")

## 4. Optimizar Temperatura Global

In [ ]:
calib_df = pd.read_csv(OUTPUT_DIR / 'calib_detections.csv')

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def nll_loss(T, logits, labels):
    """Negative Log-Likelihood con temperatura T"""
    T = max(T, 0.01)  # Evitar división por cero
    probs = sigmoid(logits / T)
    probs = np.clip(probs, 1e-7, 1 - 1e-7)
    nll = -np.mean(labels * np.log(probs) + (1 - labels) * np.log(1 - probs))
    return nll

logits = calib_df['logit'].values
labels = calib_df['is_tp'].values

# NLL antes de calibrar
nll_before = nll_loss(1.0, logits, labels)
print(f"NLL antes (T=1.0): {nll_before:.4f}")

# Optimizar T
result = minimize(lambda T: nll_loss(T, logits, labels), x0=1.0, bounds=[(0.01, 10.0)], method='L-BFGS-B')
T_optimal = result.x[0]
nll_after = result.fun

print(f"\nT óptimo: {T_optimal:.4f}")
print(f"NLL después: {nll_after:.4f}")
print(f"Mejora en NLL: {nll_before - nll_after:.4f}")

# Guardar temperatura
with open(OUTPUT_DIR / 'temperature.json', 'w') as f:
    json.dump({'T_global': T_optimal, 'nll_before': nll_before, 'nll_after': nll_after}, f, indent=2)

print(f"\nTemperatura guardada en: {OUTPUT_DIR / 'temperature.json'}")

## 5. Evaluar en val_eval

In [ ]:
with open(OUTPUT_DIR / 'temperature.json', 'r') as f:
    temp_data = json.load(f)
    T_optimal = temp_data['T_global']

val_eval_json = Path('../data/bdd100k_coco/val_eval.json')
coco_eval = COCO(str(val_eval_json))
img_ids_eval = coco_eval.getImgIds()

print(f"Procesando {len(img_ids_eval)} imágenes de val_eval...")

eval_data = []

for img_id in tqdm(img_ids_eval):
    img_info = coco_eval.loadImgs(img_id)[0]
    img_path = image_dir / img_info['file_name']
    
    if not img_path.exists():
        continue
    
    predictions = run_inference_on_image(model, img_path, TEXT_PROMPT, 0.25, CONFIG['device'])
    
    gt_anns = coco_eval.loadAnns(coco_eval.getAnnIds(imgIds=img_id))
    
    matched = match_predictions_to_gt(predictions, gt_anns, CONFIG['iou_matching'])
    eval_data.extend(matched)

eval_df = pd.DataFrame(eval_data)
eval_df.to_csv(OUTPUT_DIR / 'eval_detections.csv', index=False)

print(f"\nTotal detecciones: {len(eval_df)}")
print(f"TP: {eval_df['is_tp'].sum()}, FP: {len(eval_df) - eval_df['is_tp'].sum()}")
print(f"Guardado en: {OUTPUT_DIR / 'eval_detections.csv'}")

## 6. Calcular Métricas de Calibración

In [ ]:
def compute_calibration_metrics(logits, labels, T=1.0, n_bins=10):
    """Calcula ECE, Brier y NLL"""
    probs = sigmoid(logits / T)
    probs = np.clip(probs, 1e-7, 1 - 1e-7)
    
    # NLL
    nll = -np.mean(labels * np.log(probs) + (1 - labels) * np.log(1 - probs))
    
    # Brier Score
    brier = np.mean((probs - labels) ** 2)
    
    # ECE (Expected Calibration Error)
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]
    
    ece = 0.0
    bin_data = []
    
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = (probs >= bin_lower) & (probs < bin_upper)
        prop_in_bin = np.mean(in_bin)
        
        if prop_in_bin > 0:
            accuracy_in_bin = np.mean(labels[in_bin])
            avg_confidence_in_bin = np.mean(probs[in_bin])
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
            
            bin_data.append({
                'bin_lower': bin_lower,
                'bin_upper': bin_upper,
                'confidence': avg_confidence_in_bin,
                'accuracy': accuracy_in_bin,
                'count': np.sum(in_bin)
            })
        else:
            bin_data.append({
                'bin_lower': bin_lower,
                'bin_upper': bin_upper,
                'confidence': (bin_lower + bin_upper) / 2,
                'accuracy': 0,
                'count': 0
            })
    
    return {
        'nll': nll,
        'brier': brier,
        'ece': ece,
        'bin_data': bin_data
    }

print("Funciones de calibración definidas")

In [ ]:
eval_df = pd.read_csv(OUTPUT_DIR / 'eval_detections.csv')
with open(OUTPUT_DIR / 'temperature.json', 'r') as f:
    T_optimal = json.load(f)['T_global']

logits_eval = eval_df['logit'].values
labels_eval = eval_df['is_tp'].values

# Métricas ANTES de calibrar
metrics_before = compute_calibration_metrics(logits_eval, labels_eval, T=1.0)

# Métricas DESPUÉS de calibrar
metrics_after = compute_calibration_metrics(logits_eval, labels_eval, T=T_optimal)

print("="*60)
print("MÉTRICAS DE CALIBRACIÓN EN VAL_EVAL")
print("="*60)
print(f"\n{'Métrica':<15} {'Antes':<15} {'Después':<15} {'Mejora':<15}")
print("-"*60)
print(f"{'NLL':<15} {metrics_before['nll']:<15.4f} {metrics_after['nll']:<15.4f} {metrics_before['nll']-metrics_after['nll']:<15.4f}")
print(f"{'Brier Score':<15} {metrics_before['brier']:<15.4f} {metrics_after['brier']:<15.4f} {metrics_before['brier']-metrics_after['brier']:<15.4f}")
print(f"{'ECE':<15} {metrics_before['ece']:<15.4f} {metrics_after['ece']:<15.4f} {metrics_before['ece']-metrics_after['ece']:<15.4f}")
print("="*60)

# Guardar métricas
results = {
    'T_optimal': T_optimal,
    'val_eval': {
        'before': {k: v for k, v in metrics_before.items() if k != 'bin_data'},
        'after': {k: v for k, v in metrics_after.items() if k != 'bin_data'}
    }
}

with open(OUTPUT_DIR / 'calibration_metrics.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nMétricas guardadas en: {OUTPUT_DIR / 'calibration_metrics.json'}")

## 7. Reliability Diagrams

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for idx, (T, title) in enumerate([(1.0, 'Antes de Calibrar'), (T_optimal, 'Después de Calibrar')]):
    metrics = compute_calibration_metrics(logits_eval, labels_eval, T=T)
    bin_data = metrics['bin_data']
    
    confidences = [b['confidence'] for b in bin_data if b['count'] > 0]
    accuracies = [b['accuracy'] for b in bin_data if b['count'] > 0]
    counts = [b['count'] for b in bin_data if b['count'] > 0]
    
    ax = axes[idx]
    ax.bar(range(len(confidences)), accuracies, width=0.08, alpha=0.3, color='blue', label='Accuracy')
    ax.plot(confidences, 'ro-', label='Confidence', markersize=6)
    ax.plot([0, len(confidences)-1], [0, 1], 'k--', alpha=0.5, label='Perfect Calibration')
    
    ax.set_xlabel('Confidence Bin', fontsize=12)
    ax.set_ylabel('Accuracy / Confidence', fontsize=12)
    ax.set_title(f'{title}\nECE = {metrics["ece"]:.4f}', fontsize=13, fontweight='bold')
    ax.legend(loc='upper left')
    ax.grid(alpha=0.3)
    ax.set_ylim([0, 1.05])
    
    # Añadir conteo de muestras
    for i, count in enumerate(counts):
        ax.text(i, -0.08, f'{count}', ha='center', fontsize=9, color='gray')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'reliability_diagram.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Guardado en: {OUTPUT_DIR / 'reliability_diagram.png'}")

## 8. (Opcional) Temperature Scaling por Clase

In [ ]:
calib_df = pd.read_csv(OUTPUT_DIR / 'calib_detections.csv')

T_per_class = {}
min_samples = 50  # Mínimo de muestras para optimizar T por clase

for cat in CONFIG['categories']:
    cat_data = calib_df[calib_df['category'] == cat]
    
    if len(cat_data) < min_samples:
        T_per_class[cat] = T_optimal  # Usar T global si hay pocas muestras
        print(f"{cat:<20} - Insuficientes muestras ({len(cat_data)}), usando T global: {T_optimal:.4f}")
        continue
    
    logits_cat = cat_data['logit'].values
    labels_cat = cat_data['is_tp'].values
    
    result = minimize(lambda T: nll_loss(T, logits_cat, labels_cat), x0=1.0, bounds=[(0.01, 10.0)], method='L-BFGS-B')
    T_cat = result.x[0]
    T_per_class[cat] = T_cat
    
    print(f"{cat:<20} - T: {T_cat:.4f} (n={len(cat_data)})")

# Guardar temperaturas por clase
with open(OUTPUT_DIR / 'temperature_per_class.json', 'w') as f:
    json.dump({'T_global': T_optimal, 'T_per_class': T_per_class}, f, indent=2)

print(f"\nTemperaturas por clase guardadas en: {OUTPUT_DIR / 'temperature_per_class.json'}")

In [ ]:
eval_df = pd.read_csv(OUTPUT_DIR / 'eval_detections.csv')

with open(OUTPUT_DIR / 'temperature_per_class.json', 'r') as f:
    T_data = json.load(f)
    T_per_class = T_data['T_per_class']

eval_df['T_class'] = eval_df['category'].map(T_per_class)
eval_df['prob_calibrated_class'] = sigmoid(eval_df['logit'].values / eval_df['T_class'].values)

logits_eval = eval_df['logit'].values
labels_eval = eval_df['is_tp'].values
T_class_array = eval_df['T_class'].values

metrics_class = compute_calibration_metrics(logits_eval, labels_eval, T=1.0)
probs_class = sigmoid(logits_eval / T_class_array)
probs_class = np.clip(probs_class, 1e-7, 1 - 1e-7)
nll_class = -np.mean(labels_eval * np.log(probs_class) + (1 - labels_eval) * np.log(1 - probs_class))
brier_class = np.mean((probs_class - labels_eval) ** 2)

# ECE con T por clase
bin_boundaries = np.linspace(0, 1, 11)
ece_class = 0.0
for i in range(len(bin_boundaries) - 1):
    in_bin = (probs_class >= bin_boundaries[i]) & (probs_class < bin_boundaries[i+1])
    if np.sum(in_bin) > 0:
        acc = np.mean(labels_eval[in_bin])
        conf = np.mean(probs_class[in_bin])
        ece_class += np.abs(acc - conf) * np.mean(in_bin)

print("="*60)
print("COMPARACIÓN: T Global vs T por Clase")
print("="*60)
print(f"\n{'Métrica':<15} {'T Global':<15} {'T por Clase':<15} {'Diferencia':<15}")
print("-"*60)
print(f"{'NLL':<15} {metrics_after['nll']:<15.4f} {nll_class:<15.4f} {metrics_after['nll']-nll_class:<15.4f}")
print(f"{'Brier Score':<15} {metrics_after['brier']:<15.4f} {brier_class:<15.4f} {metrics_after['brier']-brier_class:<15.4f}")
print(f"{'ECE':<15} {metrics_after['ece']:<15.4f} {ece_class:<15.4f} {metrics_after['ece']-ece_class:<15.4f}")
print("="*60)

## 9. Verificar Impacto en mAP (Detección)

In [ ]:
from pycocotools.cocoeval import COCOeval

def create_coco_results(image_dir, coco_gt, text_prompt, model, device, T=1.0):
    """Genera resultados en formato COCO con scores calibrados"""
    results = []
    cat_to_id = {cat: i+1 for i, cat in enumerate(CONFIG['categories'])}
    
    for img_id in tqdm(coco_gt.getImgIds()):
        img_info = coco_gt.loadImgs(img_id)[0]
        img_path = image_dir / img_info['file_name']
        
        if not img_path.exists():
            continue
        
        predictions = run_inference_on_image(model, img_path, text_prompt, 0.25, device)
        
        for pred in predictions:
            score_calibrated = sigmoid(pred['logit'] / T)
            
            results.append({
                'image_id': img_id,
                'category_id': cat_to_id.get(pred['category'], 1),
                'bbox': [pred['bbox'][0], pred['bbox'][1], 
                        pred['bbox'][2] - pred['bbox'][0], 
                        pred['bbox'][3] - pred['bbox'][1]],
                'score': float(score_calibrated)
            })
    
    return results

# mAP antes de calibrar (T=1.0)
print("Calculando mAP ANTES de calibrar...")
results_before = create_coco_results(image_dir, coco_eval, TEXT_PROMPT, model, CONFIG['device'], T=1.0)

if len(results_before) > 0:
    coco_dt_before = coco_eval.loadRes(results_before)
    coco_eval_obj_before = COCOeval(coco_eval, coco_dt_before, 'bbox')
    coco_eval_obj_before.evaluate()
    coco_eval_obj_before.accumulate()
    print("\n--- mAP ANTES ---")
    coco_eval_obj_before.summarize()
    map_before = coco_eval_obj_before.stats[0]
else:
    map_before = 0.0

# mAP después de calibrar
print("\n\nCalculando mAP DESPUÉS de calibrar...")
results_after = create_coco_results(image_dir, coco_eval, TEXT_PROMPT, model, CONFIG['device'], T=T_optimal)

if len(results_after) > 0:
    coco_dt_after = coco_eval.loadRes(results_after)
    coco_eval_obj_after = COCOeval(coco_eval, coco_dt_after, 'bbox')
    coco_eval_obj_after.evaluate()
    coco_eval_obj_after.accumulate()
    print("\n--- mAP DESPUÉS ---")
    coco_eval_obj_after.summarize()
    map_after = coco_eval_obj_after.stats[0]
else:
    map_after = 0.0

print(f"\n{'='*60}")
print(f"mAP antes:   {map_before:.4f}")
print(f"mAP después: {map_after:.4f}")
print(f"Diferencia:  {map_after - map_before:.4f}")
print(f"{'='*60}")

## 10. Análisis de Distribución de Confianza

In [ ]:
eval_df = pd.read_csv(OUTPUT_DIR / 'eval_detections.csv')

prob_before = sigmoid(eval_df['logit'].values)
prob_after = sigmoid(eval_df['logit'].values / T_optimal)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribución antes
axes[0].hist(prob_before[eval_df['is_tp']==1], bins=30, alpha=0.6, label='TP', color='green', density=True)
axes[0].hist(prob_before[eval_df['is_tp']==0], bins=30, alpha=0.6, label='FP', color='red', density=True)
axes[0].set_xlabel('Confidence', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].set_title('Antes de Calibrar', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Distribución después
axes[1].hist(prob_after[eval_df['is_tp']==1], bins=30, alpha=0.6, label='TP', color='green', density=True)
axes[1].hist(prob_after[eval_df['is_tp']==0], bins=30, alpha=0.6, label='FP', color='red', density=True)
axes[1].set_xlabel('Confidence', fontsize=12)
axes[1].set_ylabel('Density', fontsize=12)
axes[1].set_title(f'Después de Calibrar (T={T_optimal:.3f})', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'confidence_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Guardado en: {OUTPUT_DIR / 'confidence_distribution.png'}")

## 11. Risk-Coverage Curves

In [ ]:
def compute_risk_coverage(scores, labels):
    """Calcula risk-coverage curve ordenando por confianza descendente"""
    order = np.argsort(scores)[::-1]
    scores_sorted = scores[order]
    labels_sorted = labels[order]
    
    coverages = []
    risks = []
    
    for i in range(1, len(scores_sorted) + 1):
        coverage = i / len(scores_sorted)
        risk = 1 - np.mean(labels_sorted[:i])
        coverages.append(coverage)
        risks.append(risk)
    
    return coverages, risks

eval_df = pd.read_csv(OUTPUT_DIR / 'eval_detections.csv')

# Antes de calibrar
scores_before = sigmoid(eval_df['logit'].values)
labels = eval_df['is_tp'].values
cov_before, risk_before = compute_risk_coverage(scores_before, labels)

# Después de calibrar
scores_after = sigmoid(eval_df['logit'].values / T_optimal)
cov_after, risk_after = compute_risk_coverage(scores_after, labels)

# Plotear
plt.figure(figsize=(10, 6))
plt.plot(cov_before, risk_before, 'b-', label='Antes (T=1.0)', linewidth=2)
plt.plot(cov_after, risk_after, 'r-', label=f'Después (T={T_optimal:.3f})', linewidth=2)
plt.xlabel('Coverage (fracción de predicciones retenidas)', fontsize=12)
plt.ylabel('Risk (tasa de error)', fontsize=12)
plt.title('Risk-Coverage Trade-off', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'risk_coverage.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Guardado en: {OUTPUT_DIR / 'risk_coverage.png'}")

# Calcular AUC
auc_before = np.trapz(risk_before, cov_before)
auc_after = np.trapz(risk_after, cov_after)
print(f"\nAUC Risk-Coverage antes:  {auc_before:.4f}")
print(f"AUC Risk-Coverage después: {auc_after:.4f}")
print(f"Mejora (menor es mejor): {auc_before - auc_after:.4f}")

## 12. Reporte Final y Resumen

In [ ]:
with open(OUTPUT_DIR / 'calibration_metrics.json', 'r') as f:
    metrics = json.load(f)

report = f"""
{'='*70}
FASE 4: TEMPERATURE SCALING - REPORTE FINAL
{'='*70}

1. TEMPERATURA ÓPTIMA
   - T global: {T_optimal:.4f}

2. MÉTRICAS DE CALIBRACIÓN EN VAL_EVAL
   
   Métrica         | Antes (T=1.0)  | Después        | Mejora
   ----------------|----------------|----------------|---------------
   NLL             | {metrics_before['nll']:14.4f} | {metrics_after['nll']:14.4f} | {metrics_before['nll']-metrics_after['nll']:14.4f}
   Brier Score     | {metrics_before['brier']:14.4f} | {metrics_after['brier']:14.4f} | {metrics_before['brier']-metrics_after['brier']:14.4f}
   ECE             | {metrics_before['ece']:14.4f} | {metrics_after['ece']:14.4f} | {metrics_before['ece']-metrics_after['ece']:14.4f}

3. INTERPRETACIÓN
   - T > 1: Modelo sobreconfidente → Temperature Scaling reduce confianza
   - T < 1: Modelo subconfidente → Temperature Scaling aumenta confianza
   - T ≈ 1: Modelo ya bien calibrado
   
   En este caso: T = {T_optimal:.4f}
   {'→ El modelo era SOBRECONFIDENTE' if T_optimal > 1 else '→ El modelo era SUBCONFIDENTE' if T_optimal < 1 else '→ El modelo ya estaba calibrado'}

4. IMPACTO EN DETECCIÓN
   - mAP antes:  {map_before:.4f}
   - mAP después: {map_after:.4f}
   - Diferencia: {map_after - map_before:+.4f}
   
   → La calibración {'MEJORA' if map_after >= map_before else 'REDUCE LIGERAMENTE'} el rendimiento de detección

5. ARTEFACTOS GENERADOS
   ✓ {OUTPUT_DIR / 'temperature.json'}
   ✓ {OUTPUT_DIR / 'calib_detections.csv'}
   ✓ {OUTPUT_DIR / 'eval_detections.csv'}
   ✓ {OUTPUT_DIR / 'calibration_metrics.json'}
   ✓ {OUTPUT_DIR / 'reliability_diagram.png'}
   ✓ {OUTPUT_DIR / 'confidence_distribution.png'}
   ✓ {OUTPUT_DIR / 'risk_coverage.png'}
   {'✓ ' + str(OUTPUT_DIR / 'temperature_per_class.json') if (OUTPUT_DIR / 'temperature_per_class.json').exists() else ''}

6. CONCLUSIONES PARA ADAS
   - Las probabilidades calibradas son más coherentes con la realidad
   - Cuando el modelo dice p=0.8, ahora acierta ~80% de las veces
   - Esto permite tomar decisiones de confianza más informadas
   - Crucial para aplicaciones de seguridad donde la confianza importa

{'='*70}
FASE 4 COMPLETADA EXITOSAMENTE
{'='*70}
"""

print(report)

with open(OUTPUT_DIR / 'final_report.txt', 'w') as f:
    f.write(report)

print(f"\nReporte guardado en: {OUTPUT_DIR / 'final_report.txt'}")

## 13. Análisis de Calibración por Clase

In [ ]:
eval_df = pd.read_csv(OUTPUT_DIR / 'eval_detections.csv')

per_class_metrics = []

for cat in CONFIG['categories']:
    cat_data = eval_df[eval_df['category'] == cat]
    
    if len(cat_data) < 10:
        continue
    
    logits_cat = cat_data['logit'].values
    labels_cat = cat_data['is_tp'].values
    
    # Antes
    metrics_cat_before = compute_calibration_metrics(logits_cat, labels_cat, T=1.0)
    
    # Después (T global)
    metrics_cat_after = compute_calibration_metrics(logits_cat, labels_cat, T=T_optimal)
    
    per_class_metrics.append({
        'category': cat,
        'n_samples': len(cat_data),
        'nll_before': metrics_cat_before['nll'],
        'nll_after': metrics_cat_after['nll'],
        'ece_before': metrics_cat_before['ece'],
        'ece_after': metrics_cat_after['ece'],
        'brier_before': metrics_cat_before['brier'],
        'brier_after': metrics_cat_after['brier']
    })

df_per_class = pd.DataFrame(per_class_metrics)
df_per_class['nll_improvement'] = df_per_class['nll_before'] - df_per_class['nll_after']
df_per_class['ece_improvement'] = df_per_class['ece_before'] - df_per_class['ece_after']

df_per_class.to_csv(OUTPUT_DIR / 'calibration_per_class.csv', index=False)

print("MÉTRICAS DE CALIBRACIÓN POR CLASE\n")
print(df_per_class.to_string(index=False))
print(f"\nGuardado en: {OUTPUT_DIR / 'calibration_per_class.csv'}")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

cats = df_per_class['category'].values
x_pos = np.arange(len(cats))

# ECE
axes[0].bar(x_pos - 0.2, df_per_class['ece_before'], 0.4, label='Antes', alpha=0.7, color='blue')
axes[0].bar(x_pos + 0.2, df_per_class['ece_after'], 0.4, label='Después', alpha=0.7, color='red')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(cats, rotation=45, ha='right')
axes[0].set_ylabel('ECE', fontsize=12)
axes[0].set_title('ECE por Clase', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3, axis='y')

# NLL
axes[1].bar(x_pos - 0.2, df_per_class['nll_before'], 0.4, label='Antes', alpha=0.7, color='blue')
axes[1].bar(x_pos + 0.2, df_per_class['nll_after'], 0.4, label='Después', alpha=0.7, color='red')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(cats, rotation=45, ha='right')
axes[1].set_ylabel('NLL', fontsize=12)
axes[1].set_title('NLL por Clase', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'calibration_per_class.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Gráfica guardada en: {OUTPUT_DIR / 'calibration_per_class.png'}")